# A Survey on Deep Neural Network Pruning: Taxonomy, Comparison, Analysis, and Recommendations

##  Типы pruning по структуре: (unstructired, structured, semi-structured)

### 1) Unstructured pruning

**Определение 1 (unstructured pruning)**. При заданных весах нейронной сети $W = \{w_0, w_1, ..., w_K\}$, наборе данных $D = \{(xi , yi)\}^N_{i=1}$, состоящем из входных $(x_i)$, выходных $(y_i)$ пар, и желаемом общем количестве ненулевых весов $k$ (меньше $K$), unstructured pruning можно записать как следующую задачу ограниченной оптимизации

$$
\min\limits_W L(W;D) = \min\limits_W \frac{1}{N}\sum\limits_{i=1}^Nl(W;(x_i,y_i)) \ (1) \\
\|\mathbf{W}\|_0 \leq k < K. 
$$
$\|\mathbf{.}\|_0$ - $L_0$ норма, $l$ - стандартная функция потерь.

**Замечание:** $L_0$ - норма, показывающая количество ненулевых элементов 

На практике для малых и средних моделей неструктурированная обрезка обычно не устанавливает веса напрямую равными 0, а устанавливает соответствующие маски (или индикаторы) $M$ в 0. В этом случае неструктурированная обрезка рассматривается как применение бинарной маски к каждому весу. Следовательно, уравнение (1) соответственно изменяется следующим образом:

$$
\min_{W, M} \mathcal{L}(W \odot M; D) = 
\min_{W, M} \frac{1}{N} \sum_{i=1}^{N} \ell(W \odot M; (x_i, y_i)), 
\quad \text{s.t.} \quad \|M\|_0 \leq k.
$$

Как правило, сеть переобучается (т.е. тонкая настройка или обучение с нуля) с фиксированными масками $М$, а замаскированные веса не участвуют в переобучении. В случае больших моделей, таких как LLM, назначение маски для каждого груза представляет собой проблему из-за огромного количества весов. Обычно веса, которые необходимо обрезать, напрямую устанавливаются в ноль. На рисунке ниже (слева) приведен пример обрезки по весам путем непосредственного удаления весов (как показано на рисунке ниже (слева)) или маскировки весов соответствующими масками (как показано на рисунке ниже (справа)), соответственно. Поскольку он может убирать грузы в любом месте, нерегулярная замена ненулевых грузов приводит к реальному ускорению, требующему поддержки специального программного и/или аппаратного обеспечения. Поэтому мы классифицируем неструктурированную обрезку как специфическую технику ускорения.

![unstructured-pruning](./pictures/Unstructured-pruning.png)

### 2) Structured pruning

**Определение 2 (Structured prunung).** Задан определенный коэффициент обрезки и нейронная сеть с $S = \{s_1, s_2, ..., s_L\}, где $s_i$ может быть набором каналов, фильтров, нейронов или transformer attention в слое $i$. Структурированная обрезка направлена на поиск $S' = \{s_1' , s_2' , ..., s_L'\}$ для минимизации снижения производительности и максимального увеличения скорости при заданном соотношении обрезки, где $s_i' \subseteq s_i , i \in \{1, .., L\}$. Структурированная обрезка удаляет целые фильтры, каналы, transformer attention или даже слои, как показано на рисунках ниже. Он не требует поддержки специального аппаратного и программного обеспечения (например, библиотек разреженной свертки) и может напрямую ускорять сети и уменьшать размер нейронных сетей.

![structured-pruning](./pictures/Structured-pruning.png)

### 3) Semi-structured pruning

Чтобы повысить гибкость структурированной обрезки и достичь более низкого падения точности при высокой скорости обрезки, в некоторых работах вводится полуструктурированная обрезка, также называемая шаблонной обрезкой, для одновременного достижения высокой точности и структурной регулярности. Некоторые примеры приведены на рисунке ниже. Например, Meng et al рассматривают фильтр как несколько полос и предлагают обрезать полосы в каждом фильтре. SparseGPT вводит шаблон разреженности 2:4 или 4:8 для уменьшения параметров LLM вдвое. В этой конфигурации по крайней мере два нуля являются обязательными в каждом наборе из четырех последовательных значений. Шаблон 2:4 может использовать разреженные тензорные ядра графического процессора NVIDIA Ampere для ускорения матричного умножения. В отличие от, structured prunning, который классифицируется как крупнозернистая обрезка, полуструктурированная обрезка классифицируется как мелкозернистая.

##  When to prune
### 1) Pruning before trainning
![before-trainning](./pictures/Pruning-before-trainning.png)
#### SNIP (SINGLE-SHOT NETWORK PRUNING BASED ON CONNECTION SENSITIVITY)
Поскольку мы намерены измерять важность (или чувствительность) каждого соединения независимо от его веса, мы вводим вспомогательные индикаторные переменные $c \in \{0, 1\}^m$, представляющие связность параметров $w$. Теперь, учитывая уровень разреженности $k$, уравнение 1 может быть соответственно модифицировано следующим образом:

$$
\min_{w, c} \mathcal{L}(w \odot c; D) = 
\min_{w, c} \frac{1}{N} \sum_{i=1}^{N} \ell(w \odot c; (x_i, y_i)) \\
\quad \text{s.t.} \quad \|c\|_0 \leq k, w \in R^m.
$$

Поскольку мы отделили вес связи (w) от наличия или отсутствия связи (c), мы можем определить важность каждой связи, измерив ее влияние на функцию потерь.

Значение $c_j$ указывает, является ли соединение $j$ активным $(c_j = 1)$ в сети или обрезанным $(c_j = 0)$. Следовательно, чтобы измерить влияние связи $j$ на потери, можно попытаться измерить разницу потерь при $c_j = 1$ и $c_j = 0$, сохраняя при этом все остальное постоянно. В точности, эффект от удаления соединения $j$ можно измерить посредством:

$$
\delta L_j(w;D)=L(1\odot w; D) - L((1 - e_j) \odot w; D)
$$

$e_j$ - индикатор элемента $j$ (на $j$ стоит 1, остальные 0), $1$ - диничный вектор


$$
\Delta L_j(\mathbf{w}; \mathcal{D}) \approx g_j(\mathbf{w}; \mathcal{D}) =
\frac{\partial L(c \odot \mathbf{w}; \mathcal{D})}{\partial c_j} \bigg|_{c=1} =
\lim_{\delta \to 0} \frac{L(c \odot \mathbf{w}; \mathcal{D}) - L((c - \delta e_j) \odot \mathbf{w}; \mathcal{D})}{\delta} \bigg|_{c=1}.
$$

В частности, наш интерес заключается в том, чтобы обнаружить важные (или чувствительные) связи в архитектуре, чтобы мы могли отсечь неважные за один раз, отделив процесс обрезки от циклов итеративной оптимизации. Для этого в качестве критерия значимости мы берем величину производных gj. Заметим, что если величина производной велика (независимо от знака), то это по существу означает, что связь cj оказывает значительное влияние на потери (как положительные, так и отрицательные), и ее необходимо сохранить, чтобы можно было обучаться на wj . Основываясь на этой гипотезе, мы определяем чувствительность связи как нормированную величину производных:

$$
s_j = \frac{|g_j(w;D)|}{\sum\limits_{k=1}^m|g_k(w;D)|}
$$

Далее выбираем $k$-лучших

![results](./pictures/SNIPresults.png)

### Pruning during trainning

Pruning during trainning (PDT) обычно использует случайно инициализированную плотную сеть $f(x; W_0)$ в качестве входной модели и совместно обучает и обрезает нейронную сеть путем обновления весов $W$ и масок весов (или фильтров, каналов и т.д.) М во время тренировки. Эти динамические схемы изменяют маски и создают подсеть $f(x; W_t \odot M_t)$ после $t$ итераций/эпох. После обрезки многие методы PDT напрямую получают подсети без необходимости обучения с нуля или тонкой настройки. 
![pruning-during-trainning](./pictures/Pruning-during-trainning.png)

#### SSS (Sparse Structure Selection)

**Обозначения**. Рассмотрим веса сверточного слоя $l$ в слоях $L$ CNN как 4-мерный тензор $W^l \in R^{N_1 \times M_l \times H_l \times W_l }$, где $N_l$ — количество выходных каналов, $M_l$ — количество входных каналов, $H_l$ и $W_l$ — высота и ширина двумерного ядра. Затем мы можем использовать $W_l^k$ для обозначения веса $k$-го нейрона в слое $l$. Коэффициенты масштабирования представлены в виде 1-мерного вектора $\lambda \in R^s$ , где S — количество структур, которые мы рассматриваем для обрезки. $\lambda^i$ относится к $i$-му значению $\lambda$. Обозначаем оператор мягкого порога как $S_{\alpha}(z)_i = sign(z_i)(|z_i | − α)_+$

**1) Sparce Structure Selection**

Если дан обучающий набор, состоящий из $N$ пар выборка-метка $\{x_i , y_i\}_{1 \leq i \leq N}$, то $L$ слоев CNN может быть представлен как функция $C(x_i ,W)$, где $W = \{W_l\}_{1 \leq l \leq L}$ представляет коллекцию всех весов в CNN. $W$ обучается через решение задачи оптимизации вида:

$$
\min_{\mathbf{W}} \frac{1}{N} \sum_{i=1}^{N} \mathcal{L}(y_i, \mathcal{C}(\mathbf{x}_i, \mathbf{W})) + \mathcal{R}(\mathbf{W}),
$$

$\mathcal{L}(y_i, \mathcal{C}(\mathbf{x}_i, \mathbf{W}))$ - потеря на выборке $x_i$; $\mathcal{R}(.)$ - неструктурированная регуляризация, применяемая к каждому весу, например, $l_2$-норма как распад веса. Исходя из уравнения ма пока можем обнулять каждый вес.

В отличие от прямого обнуления весов в одной группе, мы пытаемся принудительно обнулить выход группы. Для достижения этой цели мы вводим новый тип параметра – коэффициент масштабирования $\lambda$ для масштабирования выходов некоторых конкретных структур (нейронов, групп или блоков), а также добавляем ограничение разреженности на $\lambda$ во время обучения. Наша цель состоит в том, чтобы получить разреженное $\lambda$. А именно, если $\lambda^i=0$, то мы можем смело удалять соответствующую структуру, так как ее выходы не вносят никакого вклада в последующие вычисления.

Формально целевую функцию предлагаемого нами метода можно сформулировать следующим образом:

$$
\min\limits_{W, \lambda} \frac{1}{N} \sum_{i=1}^{N} \mathcal{L}(y_i, \mathcal{C}(\mathbf{x}_i, \mathbf{W}, \lambda)) + \mathcal{R}(\mathbf{W}) + \mathcal{R}_s(\lambda)
$$

где $\mathcal{R}_s(.)$ — регуляризация разреженности для $\lambda$ с весом $\gamma$. В данной работе мы рассматриваем наиболее часто используемую ее выпуклую релаксацию $l_1$-нормы, которая определяется как $\gamma \|\mathbf{W}\|_1$

Для $W$ мы можем обновить его с помощью SGD. Для $\lambda$ мы используем метод ускоренного проксимального градиента (APG). Для лучшей иллюстрации мы перепишем задачу в виде:

$$
\min\limits_{\lambda}G(\lambda) + mathbf{R}_s(\lambda)
$$

Затем мы можем обновить $\lambda$ на APG momentum base:

$$
d_{(t)} = \lambda_{(t-1)} + \frac{t-2}{t+1}(\lambda_{(t-1)} - \lambda_{(t-2)}) \\
z_{(t)} = d_{(t)} - \eta_{(t)}\nabla G(d_{(t)}) \\ 
\lambda_{(t)} = prox_{\eta_{(t)}\mathbf{R}_s}(z_{(t)})
$$

$\eta_{(t)}$ - шаг градиента на итерации $t$;  

**Замечание:**

Accelerated Proximal Gradient (APG) - метод оптимизации, который используется для решения задач выпуклой оптимизации, особенно когда целевая функция включает в себя сумму дифференцируемой выпуклой функции и недифференцируемого терма. APG основан на идеях ускоренных градиентных методов Нестерова.

Решает задачи вида:
$$
\min\limits_x\{F(x)=f(x) + g(x)\}
$$

$f(x)$ - дифференцируемая выпуклая функция с 𝐿-липшицевым градиентом;\\ $g(x)$ - выпуклая, но потенциально недифференцируемая функция, например, $g(x) = \lambda \|\mathbf{x}\|$ для задач с регуляризацией $L_1$.

$$
prox_{\alpha g}(z) = arg \min\limits_x (g(x) + \frac{1}{2\alpha}\|\mathbf{x-z}\|^2)
$$


$prox_{\eta_{(t)}\mathbf{R}_s}(.) = S_{\eta \gamma}(.)$ т.к. $\mathbf{R}_s(\lambda)=\gamma \|\mathbf{\lambda}\|_1$

**2) Example Block Selection**

Формально остаточный блок с сопоставлением тождеств может быть сформулирован по следующей формуле:

$$
r^{i+1} = r^i + F^i(r^i, W^i)
$$

$r^i$ и $r^{i+1}$ — вход и выход $i$-го блока; $F^i$ — остаточная функция и $W_i$ — параметры блока.

Чтобы обрезать блоки, мы добавляем коэффициент масштабирования после каждого остаточного блока. Тогда в наших рамках формулировка уравнения выше является следующей:

$$
r^{i+1} = r^i + \lambda^i F^i(r^i, W^i)
$$

Те, где $\lambda^i$ будет равно 0 - блоки будут убраны

### Pruning after trainning

![pruning](./pictures/pruning-after-trainning.png)

Pruning after trainning (PAT) является наиболее популярным типом конвейера обрезки, поскольку обычно считается, что предварительное обучение плотной сети необходимо для получения эффективной подсети. Особенно для больших моделей, таких как LLM и диффузионные модели, обрезка специально применяется к предварительно обученным моделям. Этот класс методов обрезки обычно следует процессу Pretrain-Prune-Retrain или Pretrain-Prune, как показано на рисунке выше, состоит из 3-х этапов:
1) Предварительно обучить случайно инициализированную плотную сеть $f(x; W_0)$ сходиться к $f(x; W_t)$.
2) Отрежьте весовые коэффициенты (или фильтры, нейроны и т.д.), которые оказывают наименьшее влияние на производительность, и настройте отсеченную сеть $f(x; W_t' \odot M')$ для нескольких итераций, где W_t' M' — веса и маски после обрезки соответственно. Этот шаг может быть выполнен как минимум один раз (т.е. одномоментная обрезка) или несколько раз (т.е. итеративная обрезка).
3) Тренируются отсавшиеся веса или fine-tune

Разреженность может постепенно увеличиваться в процессе обрезки до тех пор, пока не будет достигнута цель.

#### SparceGPT



**1) Fast Approximate and Reconstruction**

**Идея.**

Запишем задачу в следующем виде:

$$
arg \min\limits_{M_l, W} \|\mathbf{W_lX_l - (M_l \odot \^W_l)X_l}\|_2^2
$$

$l$ - номер слоя; $\^W_l$ - возможно обновленные веса; $M_l$ - маска с определенной целевой плотностью

Для фиксированной маски обрезки M оптимальные значения всех весов в маске могут быть точно рассчитаны путем решения задачи разреженной реконструкции, соответствующей каждой строке матрицы $w_i$ через:

$w^i_{M_i} = (X_{M_i}X_{M_i}^T)X_{M_i}(w_{M_i}X_{M_i})^T$

$XM_i$ обозначает только подмножество входных объектов, соответствующие веса которых не были обрезаны в строке $i$, а $w_{M_i}$ представляет их соответствующие веса. Однако для этого необходимо инвертировать гессенову матрицу $H_{M_i} = X_{M_i}X_{M_i}^T$, соответствующую значениям, сохраняемым маской обрезки $M_i$ для строки $i$, т.е. вычисления (H_{M_i})^{-1}, отдельно для каждой строки. Это очень дорого.

**Замечание:** $(H_{M_i})^{-1} \neq (H^{-1})_{M_i}$

![hessian](./pictures/Hessian.png)

**Замечание:** Почему $H_{M_i} = X_{M_i}X_{M_i}^T$?

$$
L(w)= \|\mathbf{wX-b}\|^2 = (wX-b)(wX-b)^T \\

\frac{\partial L}{\partial w} = 2X(wX - b)^T \\

H = \frac{\partial^2 L}{\partial w \partial w^T} = 2X X^T
$$

**Используем другой способ: OBS(Optimal Brain Surgeon):**

Разложим в ряд Тейлора функцию ошибки pruning:

$$
\delta E = (\frac{\partial E}{\partial w})^T\delta w^T + \frac{1}{2}\delta w^T H \delta w + O(|\mathbf{\delta w}\|^3) \\
H = \frac{\partial^2 E}{\partial w^2}
$$ 

Пусть мы хотим обнулить какое-то значение $w_q$ в строке, тогда нам необходимо использовать следующую формулу:

$$
e^T_q \delta w + w_q = 0
$$

Чтобы это сделать с минимальной ошибкой нам необходимо занулить второй член ряда тейлора, тогда нашу задачу можно переписать в следующем виде:

$$
\min\limits_q\{\min\limits_{\delta w}^T(\frac{1}{2}\delta w^T H \delta w) | e^T_q \delta w + w_q = 0\}
$$

Перепишем эту задачу в виде Лангражинана и будем решать задачу поиска локального минимума при заданных условиях:

$$
L = \frac{1}{2}\delta w^T H \delta w + \lambda (e^T_q \delta w + w_q)
$$

Его минимум можно найти в следующем виде:

$$
\delta w = -\frac{w_q}{[H^{-1}]_{qq}}H^{-1}e_q \\
L_q = \frac{1}{2}\frac{w_q^2}{[H^{-1}]_{qq}}
$$


Но проблема все равно остается в подсчете $H$ и обратной ей матрицы. Пусть $o=F(w,in)$ - выход нейронной сети и будем минимизировать среднеквадратичную ошибку:

$$
E = \frac{1}{2P}\sum\limits_{k=1}^P(t^{[k]} - o^{[k]})^T(t^{[k]} - o^{[k]})
$$
Тогда:

$$
H = \frac{\partial^2 E}{\partial w^2} = \frac{1}{P}\sum\limits_{k=1}^P [\frac{\partial F(w, in^{[k]})}{\partial w} \frac{\partial F(w, in^{[k]})^T}{\partial w} - \frac{\partial F(w, in^{[k]})}{\partial w^2} (t^{[k]} - o^{[k]})]
$$

Далее мы рассмотрим сеть, полностью обученную до минимума ошибок при $w^{*}$. При том условии, что сетевой ответ будет близок к
желаемому отклику t, и, следовательно, мы можем пренебресь вторым слагаемым. Такое приближение может быть оправдано даже на поздних этапах обработки, когда эта ошибка не мала для отдельного шаблона.

$$
H = \frac{1}{P}\sum\limits_{k=1}^P [\frac{\partial F(w, in^{[k]})}{\partial w} \frac{\partial F(w, in^{[k]})^T}{\partial w}]
$$
Пусть $X^{[k]}=\frac{\partial F(w, in^{[k]})}{\partial w}$:

$$
H = \frac{1}{P}\sum\limits_{k=1}^P X^{[k]} X^{[K]T} = \frac{1}{P}\sum\limits_{k=1}^P \sum\limits_{l=1}^{n_0}X_l^{[k]}X_l^{[k]T}
$$
$n_0$ - размерность вектора (матрицу представляем в виде вектора)

Исходя из этой формулы мы можем вычислять $H$ итеративно:

$$
H_{m+1} = H_m + \frac{1}{P}X^{[m+1]}X^{[m+1]T}
$$

Исходя из формулы: 

$$
(A + BCD)^{-1} = A^{-1}-A^{-1}B(C^{-1}+DA^{-1}B)^{-1}DA^{-1}
$$

обратную матрицу можно вычислить по формуле:

$$
H^{-1}_{m+1} = H^{-1}_{m} - \frac{H^{-1}_mX^{[m+1]X^{[m+1]T}H^{-1}_m}}{P + X^{[m+1]T}H^{-1}_mX^{[m+1]}} \\
H^{-1}_0 = \alpha^{-1}I, \ \alpha \in (10^{-8}, .. , 10^{-4})
$$

**SparceGPT (Optimal Partial Updates)**

Применение обновления OBS δm потенциально корректирует значения всех доступных параметров (в текущей маске $M$) с целью компенсации удаления wm. Однако, что, если мы обновим веса только в подмножестве $U \subseteq M$ среди оставшихся необрезанных весов? Таким образом, мы все еще можем извлечь выгоду из компенсации ошибок, используя только веса в $U$, при этом снижая стоимость применения OBS.

Такое частичное обновление действительно может быть достигнуто путем простого вычисления обновления OBS с использованием $H_U$, гессенского значения, соответствующего $U$, а не $H_M$, и обновления только $w_U$. Важно отметить, что потеря нашей конкретной задачи по слоям остается квадратичной, и обновления OBS по-прежнему оптимальны: ограничение до $U$ само по себе не влечет за собой никакой дополнительной аппроксимационной ошибки, только компенсация ошибки может быть не такой эффективной, так как доступно меньше весов для настройки. В то же время, если $|U| < |M|$, то инвертирование $H_U$ будет намного быстрее, чем инвертирование $H_M$. Теперь мы будем использовать этот механизм для достижения нашей цели по синхронизации гессенцев в масках во всех рядах $W$.

**Hessian Synchronization**

Далее предположим фиксированный порядок входных признаков $j = 1,..., d_{col}$. Поскольку они обычно располагаются случайным образом, мы просто сохраним данный порядок для простоты, но в принципе можно выбрать любую перестановку. Далее мы рекурсивно определяем последовательность подмножеств индекса $d_{col} $U_j$ следующим образом:

$$
U_{j+1} = U_j = \{j\}, \ U_1 = \{1,...d_{col}\}
$$

Другими словами, начиная с $U_1$, являющегося множеством всех индексов, каждое подмножество $U_{j+1}$ создается путем удаления наименьшего индекса из предыдущего подмножества $U_j$. Эти подмножества также накладывают последовательность обратных гессенцев $(H_{U_j})^{−1} = ((XX^T)U_j )^{−1}$, которую мы собираемся разделить на все строки W. Важно отметить, что $(H_{U_j+1})^{−1}$ можно эффективно вычислить, удалив первую строку и столбец, соответствующие $j$ в исходном $H$,  от $B = (H_{U_j})^{−1}$:

$$
(H_{U_{j+1}})^{-1} = (B - \frac{1}{[B]_{11}}B_{:,1B_{1,:}})_{2:,2:}
$$

После того, как вес был обрезан, его больше не следует обновлять. Далее, когда мы обрезаем $w_k$, мы хотим обновить как можно больше необрезанных весов для максимальной компенсации ошибок. Это приводит к следующей стратегии: перебираем $U_j$ и соответствующие им обратные гессенцы $(H_{U_j} )^{−1}$ по порядку и обрезаем $w_j$, если $j \in M_i$, для всех строк $i$. Важно отметить, что каждое обратное значение Гессена $(H_{U_j} )^{−1}$ вычисляется только один раз и повторно используется для удаления веса $j$ во всех строках, где он является частью маски обрезки. 

Таким образом общий алгоритм работы можно проиллюстрировать следующим образом:

![algo](./pictures/OverallAlgorithm.png)


### Общая схема всех видов pruning

![all](./pictures/AllVariancePruning.png)